This notebook is used for data augmentation

In [9]:
from PIL import Image
import pandas as pd
import cv2
import os

IMAGE_FILETYPE = ".jpg"
class ImageAugmentation():
    def __init__(self, label_directory, image_directory, save_directory = None):
        self.metadata = pd.read_csv(label_directory)
        self.image_path = image_directory
        self.save_path = self.image_path if save_directory is None else save_directory
        
    def transform_one_image(self, image_metadata:pd.Series):
        """
        Applies all rotation and flip transformations (7 total) for one image

        Args:
            image_metadata: A row in the label file
            
        Reurns:
            A new data frame containing the track_ids and duplicated metadata
        """
        reps = 7
        names = image_metadata.keys()
        
        id = image_metadata["id_track"]
        
        # The augmented ids will simply have 10 through 70 prepended to the existing id
        augmented_ids = [int(f"{i + 1}0{id}") for i in range(reps)]
        
        # Attempt to read the track image
        img = cv2.imread(os.path.join(self.image_path, str(id) + IMAGE_FILETYPE)) #type:ignore
        
        # The first three are purely rotation, the remaining 4 are flipping and rotation
        augmented_images = [
            cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE),
            cv2.rotate(img, cv2.ROTATE_180),
            cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE),
            cv2.flip(img, 0),
            cv2.rotate(cv2.flip(img, 0), cv2.ROTATE_90_CLOCKWISE),
            cv2.rotate(cv2.flip(img, 0), cv2.ROTATE_180),
            cv2.rotate(cv2.flip(img, 0), cv2.ROTATE_90_COUNTERCLOCKWISE)
        ]

        # Save the augmented images
        for i in range(reps):
            cv2.imwrite(os.path.join(self.save_path, 
                                     str(augmented_ids[i]) + IMAGE_FILETYPE),
                        augmented_images[i])
            
        # Return the pandas record for augmented images
        records = dict()
        for name in names:
            if name == ["id_track"]:
                records[name] = augmented_ids
            else:
                records[name] = [image_metadata[name]] * reps
        return pd.DataFrame(records)

        
        
        
        
        
        

In [14]:
# Testing Code:
current_path = "../"
image_path = current_path + "data/images_activity"
label_path = current_path + "data/activity_label.csv"

aug = ImageAugmentation(label_path, image_path, "")
data = pd.read_csv(label_path)
aug.transform_one_image(data.iloc[1])



,id_track,activity
0,31250251,transit
1,31250251,transit
2,31250251,transit
3,31250251,transit
4,31250251,transit
5,31250251,transit
6,31250251,transit
